# Hack4Rail - Challenge 9

Sample notebook to access battery data on Snowflake

In [ ]:
from dotenv import load_dotenv
import plotly.express as px
from charged.snowflake_utils import create_session
import snowflake.snowpark.functions as F
 
# load environment variables from .env file
load_dotenv()

# create snowpark session
session = create_session()

In [ ]:
# query tables BATTERIELOK_DATA, VEHICLES, VEHICLE_TYPE
table = "BATTERIELOK_DATA"
sdf = (
    session
    .table(table)
    .filter(F.col("VEHICLE_ID").isNotNull())
    .with_column(
        "TIMESTAMP_TRUNC",
        F.from_unixtime(
            F.round(F.unix_timestamp(F.col("TIMESTAMP_VEHICLE")) / 60) * 60
        ).cast("TIMESTAMP"),
    )
)
sdf.show()

In [ ]:
# sdf.columns

metrics = [
    "VEHICLE_OUTSIDE_TEMP",
    "BATTERY_SOC",
    "BATTERY_SOH",
    "BATTERY_COOLING_TEMP",
    "BATTERY_1_TEMP",
    "BATTERY_1_VOLTAGE",
    "BATTERY_1_CURRENT",
    "BATTERY_2_TEMP",
    "BATTERY_2_VOLTAGE",
    "BATTERY_2_CURRENT",
    "BATTERY_3_TEMP",
    "BATTERY_3_VOLTAGE",
    "BATTERY_3_CURRENT",
    "BATTERY_4_TEMP",
    "BATTERY_4_VOLTAGE",
    "BATTERY_4_CURRENT",
    "BATTERY_5_VOLTAGE"
]

from_date = '2025-05-23'
to_date = '2025-06-23'

In [ ]:
# groupby vehicle and 60 sec
aggregations = []
for column in metrics:
    aggregations.append(F.avg(F.col(column)).alias(f"{column}_AVG"))

df = (
    sdf
    .filter(F.to_date(F.col("TIMESTAMP_VEHICLE")) >= from_date)
    .filter(F.to_date(F.col("TIMESTAMP_VEHICLE")) <= to_date)
    .group_by(["VEHICLE_ID", "TIMESTAMP_TRUNC"])
    .agg(*aggregations)
    .order_by(["VEHICLE_ID", "TIMESTAMP_TRUNC"], ascending=[True, True])
    .to_pandas()
)
df.head()


In [ ]:
# show 
fig = px.line(
    df,
    x="TIMESTAMP_TRUNC",
    y="BATTERY_SOC_AVG",
    markers=True,
    # color="VEHICLE_ID",
    facet_row="VEHICLE_ID",
    render_mode="svg",
)

fig.show()